# Overview

In this notebook, we will dive into LLM and their synergy with Vector Databases. The vector databases store data in a unique format known as **vector embeddings** which enable LLMs to grasp and utilize information more contextually and accurately. Let's create an application that **context-aware and reliable**. 


# Embedding Process

In deep learning, the raw data transformed into a numerical format which know as vectors that AI system can understand. High-dimensional data referes to data that has many attributes or features, each representing a different dimension. These dimensions help in capturing the nuanced characteristics of the data. The vector embeddings process likes below:

```
Raw input -> Tokenization -> Embedding -> Vector
```

Each number in vector represents a specific feature of the data, and together, these numbers encapsulate the seence of the original input int a format that the machine can process. For example, see the illustration below:

![](https://cdn.masto.host/sigmoidsocial/media_attachments/files/112/030/524/565/225/614/original/62e6da79e44eda50.webp)

The vector representation of puppy would be positioned closer in vector space to dog than to house, reflecting their semantic proximity. This approach extends to analogical relationships as well. The vector distance and direction between man and woman can be analogous to that between king and queen. This illustrates **how word vectors not only represent words but also allow for a meaningful comparison of their semantic relationships in a multidimensional vector space.**


# Vector Databases

**Similarity Seach** is core function where vector databases excel. They can quickly **find data points that are similar to given query in a high-dimensional space.** Some examples like content-based Retrival, E-Commerce image searching, etc. It is also can enhance LLMs with **Contextual Understanding**. It stores and process text embeddings that enable LLMs to perform more nuanced and context-aware information retrieval. They help in understanding


